<a href="https://colab.research.google.com/github/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/Topic_Modeling_with_BERTopic_Reclame_aqui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Topic Modeling with BERTopic - Reclame Aqui**

BERTopic is a topic modeling technique that leverages transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions 

Reference: (https://maartengr.github.io/BERTopic/index.html).

### **Enabling the GPU**

We will use the GPU provided by COLAB to accelarate our model training. To enable GPUs for the notebook:
1- Navigate to Edit -> Notebook Settings
2- Select GPU from the Hardware Accelerator drop-down

In [1]:
# verify if GPU is enable
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jan 24 17:00:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### **Github**

In [2]:
!ssh-keygen -t rsa -b 4096
# Add github.com to our known hosts
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
# Restrict the key permissions, or else SSH will complain.
!chmod go-rwx /root/.ssh/id_rsa

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 
Created directory '/root/.ssh'.
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:Fh4cdmnZsowxzRMMotLozrxRZTPj9pGx/1Acx32g94M root@34b323821257
The key's randomart image is:
+---[RSA 4096]----+
|      . +=o=  .  |
|   o . +ooX .o o |
|  o o * =* +o + o|
| . . + =.*o. +...|
|  . . o S   oE ..|
| + . . o o .    .|
|  =     . o      |
|   o       o     |
|  .         .    |
+----[SHA256]-----+
# github.com:22 SSH-2.0-babeld-f1c576c9


In [3]:
!cat /root/.ssh/id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDg0telDkpMUHervpV3rsXlSpVq+yz77IVBGZ5vvGrOKr6lmLu20o6+cOOI65KwMWuG4UXzkw1nVKB7FaNVvv/h/eUZyvM4ENVBRxHi2xS3Yb9k2Z/S2OaGAh6l18k8EUNtMfP+heAvnLT5mO/nAQQEJEGe6UEcwCK8a3HkLovYghJXAr+4k0g+APXmv/gcYfV6Cu3yF1CFCtJygKr97TIcjOPRXagMjqSiUQy4NvwbyQA1CfkjhhAepjD0UyAovLfknsqoEKYCJqq2FUU9MDJmA2diIDVu8wSAdQev3FWZbGgWw46kqHtkQQ7qjeMtq9puXkYM3olEM2wiqD2JdyOFsKrySoD2nakmn3reFVAfRv1ZCw0QrOwd9MLs5W8y093efE6wcdSieUvFY3rahva8g1qrtwhVX1Qy9ccF1j7JFepEZMDX2ujJcpgUMXmqjnIbH6iqqOgJ12uMzYk4EPyouaXqWFzcAfdIsdoPIa+91eqFeLDWZs2oyUMTQB0GDdUac6oMADz2Yp5bfCY2reSnBCBjPrfFUURG/AzZy+OWUM1m8n/siIHXqgiSmhpN0OmwJZX40cgimbSvShAZScby+FaqFDq2zDY2P8GHuQqi6LXNDhu/O0roxyXNA9abE2fmL3LzZ6CuTXMic2HoehIohKCvE9Hu6etCTLBI4dSdxw== root@34b323821257


In [4]:
!git config --global user.email ""
!git config --global user.name "punkmic"

In [5]:
!ssh -T git@github.com

Hi punkmic! You've successfully authenticated, but GitHub does not provide shell access.


In [6]:
!git clone git@github.com:punkmic/Topic-Modeling-Reclame-Aqui.git

Cloning into 'Topic-Modeling-Reclame-Aqui'...
remote: Enumerating objects: 16629, done.
remote: Counting objects: 100% (2448/2448), done.
remote: Compressing objects: 100% (2021/2021), done.
remote: Total 16629 (delta 404), reused 2425 (delta 391), pack-reused 14181
Receiving objects: 100% (16629/16629), 158.22 MiB | 7.03 MiB/s, done.
Resolving deltas: 100% (1552/1552), done.
Updating files: 100% (14939/14939), done.


### **Setup**

In [7]:
%%capture
import pandas as pd # for data manipulation
import os # for interacting with the operating system
import nltk # for natural language processing
import matplotlib.pyplot as plt # for visualization
import ast # for convert str to tuple
import csv
import json
import pickle 
from sklearn.decomposition import PCA # for dimension reduction
from sklearn.feature_extraction.text import CountVectorizer # for convert text documents to matrix of tokens count
from sklearn.cluster import KMeans # for clustering

try:
  from bertopic import BERTopic # for topic modeling
  import optuna # for hyperparameter optimization
  from hdbscan import HDBSCAN # for clustering
  from umap import UMAP # for dimension reduction
  from bertopic.vectorizers import ClassTfidfTransformer 
  from gensim import models
  from gensim.corpora import Dictionary
  from gensim.models.coherencemodel import CoherenceModel
except:
  !pip install bertopic
  !pip install kaleido # for save BERTopic plots as image
  !pip install optuna
  !pip install hdbscan
  !pip install umap-learn
  !pip install gensim
  from gensim import models
  from gensim.corpora import Dictionary
  from gensim.models.coherencemodel import CoherenceModel
  from umap import UMAP # for dimension reduction
  import optuna # for hyperparameter optimization
  from hdbscan import HDBSCAN # for clustering
  from bertopic import BERTopic # for topic modeling
  from bertopic.vectorizers import ClassTfidfTransformer 

In [34]:
WORK_DIR = '/content/Topic-Modeling-Reclame-Aqui/bertopic'

In [9]:
COLNAME = 'documents'
df = pd.read_csv(os.path.join('/content/Topic-Modeling-Reclame-Aqui/datasets', 'processed_v1.csv'))
documents = df[COLNAME].values

In [10]:
df.head(3)

,title,documents,documents_nouns,bigrams,noun_bigrams,trigrams,noun_trigrams,freq_words_removed,freq_words_removed_nouns
0,pedir cancelado justificativa semana compro,pesquisando bastante novo comprar resolver agu...,semana desconto promoção desconto compra custo...,"[('pesquisando', 'bastante'), ('bastante', 'no...","[('semana', 'desconto'), ('desconto', 'promoçã...","[('pesquisando', 'bastante', 'novo'), ('bastan...","[('semana', 'desconto', 'promoção'), ('descont...",pesquisando bastante novo resolver aguardar se...,semana desconto promoção desconto plataforma c...
1,pedir cancelar,sinceramente decepcionar entrar contato procon...,procon audiência conciliação solicitação produ...,"[('sinceramente', 'decepcionar'), ('decepciona...","[('procon', 'audiência'), ('audiência', 'conci...","[('sinceramente', 'decepcionar', 'entrar'), ('...","[('procon', 'audiência', 'conciliação'), ('aud...",sinceramente decepcionar procon hoje informar ...,procon audiência conciliação solicitação estoq...
2,cobrança indever,cancelei plano antes terminar período testir g...,cancelei período plataforma fatura gratuito ca...,"[('cancelei', 'plano'), ('plano', 'antes'), ('...","[('cancelei', 'período'), ('período', 'platafo...","[('cancelei', 'plano', 'antes'), ('plano', 'an...","[('cancelei', 'período', 'plataforma'), ('perí...",cancelei plano antes terminar período testir g...,cancelei período plataforma fatura gratuito ca...


## **Training a BERTopic Model**

The BERTopic algorithm has several advantages over other topic modeling algorithms. It is able to handle sparse data, it is scalable to large datasets, and it is able to learn topics that are not well-defined or are overlapping.

As our data language is portuguese we will going to set language to multilingual.

Create a new BERTopic model and train it. By default BERTopic use the paraphrase-multilingual-MiniLM-L12-v2 model for multi language documents. For others model check here [BERTopic sentence transformers](https://maartengr.github.io/BERTopic/getting_started/embeddings/embeddings.html#sentence-transformers)

In [128]:
# Create a new BERTopic model using multilingual option
topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True)

# Train model 
topics, probs = topic_model.fit_transform(documents)

Batches:   0%|          | 0/324 [00:00<?, ?it/s]

In [132]:
topic_range = range(0, min(len(topics), 20), 1)
top_n_topics = len(topic_range)

range(0, 20)


BERTopic works in three main steps: 


1.   Documents are first converted to numeric data. It extracts different embeddings based on the context of the word. For this, a sentence transformation model is used.
2.  Documents with similar topics are then grouped together forming clusters with similar topics. For this purpose, BERTopic uses the clustering algorithm UMAP to lower the dimensionality of the embeddings. Then the documents are clustered with the density-based algorithm HDBSCAN.
3. BERTopic extracts topics from clusters using a class-based TF-IDF score. This score gives the importance of each word in a cluster. Topics are then created based on the most important words measured by their C-TF-IDF score.

For more information check this link [BERTopic](https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6)



#### **BERTopic coherence score**

In [12]:
def get_bertopic_coherence(model, topics, docs):
    # preprocess Documents
    documents = pd.DataFrame({"Document": docs,
                          "ID": range(len(docs)),
                          "Topic": topics})
    documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
    cleaned_docs = model._preprocess_text(documents_per_topic.Document.values)

    # extract vectorizer and analyzer from BERTopic
    vectorizer = model.vectorizer_model
    analyzer = vectorizer.build_analyzer()

    # extract features for Topic Coherence evaluation
    tokens = [analyzer(doc) for doc in cleaned_docs]
    dictionary = Dictionary(tokens)
    corpus = [dictionary.doc2bow(token) for token in tokens]
    topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

    # evaluate
    coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
    return coherence_model.get_coherence()

In [13]:
print(f"Coherence score: {get_bertopic_coherence(topic_model, topics, documents)}")

Coherence score: 0.4774262142258511


### **Extracting Topics**

In [14]:
# Print the most frequent topics
freq = topic_model.get_topic_info()

# Show the top 5 most frequent topics
freq.head(5)

,Topic,Count,Name
0,-1,5688,-1_produto_entregar_pedir_compra
1,0,448,0_celular_aparelho_fone_telefone
2,1,343,1_cartão_crédito_cobrar_cobrança
3,2,218,2_previsão_entregar_data_prazo
4,3,202,3_endereço_remetente_correio_pedir


The table above shows the five most freqeuente topics and the words present on it extract by BERTopic. -1 refers to all outliers and should be ignored.

In [15]:
# show the most frequent topic
topic_model.get_topic(0)

[('celular', 0.051784902687844074),
 ('aparelho', 0.020679876806984233),
 ('fone', 0.01229101268661818),
 ('telefone', 0.011714172521142067),
 ('comprar', 0.011213595926734958),
 ('compra', 0.009229584228716815),
 ('loja', 0.008867366371452124),
 ('contato', 0.008257051568445443),
 ('entregar', 0.00800401179018565),
 ('site', 0.007960970908728923)]

**Note:** BERTopic is stocastich which means that the topics might differ across runs this is mostly due to the stocastisch nature of UMAP

**Save topic info table as CSV**

In [38]:
def save_freq_topics(model, label):
  # Print the most frequent topics
  freq = topic_model.get_topic_info()

  # Show the top 5 most frequent topics
  freq = freq.head(5)

  dir = os.path.join(WORK_DIR, COLNAME, 'frequent_topics')
  
  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(dir):
    os.makedirs(dir)

  freq.to_html(os.path.join(dir, f"freq_topics_trial_{label}.json"))
  freq.to_csv(os.path.join(dir, f"freq_topics_trial_{label}.csv"), index=False)

In [39]:
save_freq_topics(topic_model, COLNAME)

## **Visualization**

### **Intertopic Distance Map**

This graph shows the distance intertopic and help us understand the promixity of topics

In [68]:
fig = topic_model.visualize_topics(top_n_topics=20, width=800, height=800)
fig

**Save intertopic distance map**

In [133]:
def save_topics(model, label, top_n_topics=-1):
  fig = model.visualize_topics(top_n_topics=min(top_n_topics, 20), width=650, height=600)

  dir = os.path.join(WORK_DIR, COLNAME, 'topics')
  
  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(dir):
    os.makedirs(dir)

  fig.write_image(os.path.join(dir, f"intertopic_distance_map_trial_{label}.png"), format="png")
  fig.write_html(os.path.join(dir, f"intertopic_distance_map_trial_{label}.html"))

In [70]:
save_topics(topic_model, COLNAME, top_n_topics)

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/intertopic_distance_map/idm_preprocessed_lemma.png?raw=true)

### **Visualize Topic Hierarchy**

The topics that were created can be hierarchically reduced. This visualization shows how the topics relate to one another.

In [71]:
fig = topic_model.visualize_hierarchy(top_n_topics=20, width=800, height=800)
fig

**Save Hierarchical Clustering**

In [134]:
def save_hierarchy(model, label, top_n_topics=-1):
  fig = model.visualize_hierarchy(top_n_topics=min(top_n_topics, 20), width=500, height=500)

  # Set the path to save 
  dir = os.path.join(WORK_DIR, COLNAME, 'hierarchical_clusterings')

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(dir):
    os.makedirs(dir)

  fig.write_image(os.path.join(dir, f"hierarchical_clustering_trial_{label}.png"), format="png")
  fig.write_html(os.path.join(dir, f"hierarchical_clustering_trial_{label}.html"))

In [73]:
save_hierarchy(topic_model, COLNAME, top_n_topics)

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/hierarchical_clustering/hc_preprocessed_lemma.png?raw=true)

### **Visualize Terms**

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation.

In [74]:
fig = topic_model.visualize_barchart(top_n_topics=8, width=300, height=300)
fig

**Save Top Word Scores Bar Chart**

In [135]:
def save_top_words_scores(model, label, top_n_topics=-1):
  fig = model.visualize_barchart(top_n_topics=min(top_n_topics, 8), width=250, height=250)

  # Set the path to save 
  dir = os.path.join(WORK_DIR, COLNAME, 'top_words_scores')

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(dir):
    os.makedirs(dir)

  fig.write_image(os.path.join(dir, f"top_words_scores_trial_{label}.png"), format="png")
  fig.write_html(os.path.join(dir, f"top_words_scores_trial_{label}.html"))

In [76]:
save_top_words_scores(topic_model, COLNAME, top_n_topics)

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/top_words_scores/tws_preprocessed_lemma.png?raw=true)

### **Visualize Topic Similarity**

This plot shows a similarity matrix by simply applying cosine similarities through those topic embeddings generate by BERTopic through both c-TF-IDF and embeddings. This matrix indicate how similar certain topics are to each other.

In [77]:
fig = topic_model.visualize_heatmap(n_clusters=20, width=800, height=800)
fig

 **Save Similarity Matrix**

In [78]:
def save_similarity_matrix(model, label):
  fig = model.visualize_heatmap(n_clusters=10, width=800, height=800)

  # Set the path to save 
  dir = os.path.join(WORK_DIR, COLNAME, 'similarity_matrixes')

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(dir):
    os.makedirs(dir)

  fig.write_image(os.path.join(dir, f"similarity_matrix_trial_{label}.png"), format="png")
  fig.write_html(os.path.join(dir, f"similarity_matrix_trial_{label}.html"))

In [79]:
save_similarity_matrix(topic_model, COLNAME)

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/similarity_matrix/sm_preprocessed_lemma.png?raw=true)

### **Visualize Term Score Decline**

Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added.

In [83]:
fig = topic_model.visualize_term_rank(topics=range(0, 20, 1), width=800, height=500)
fig

**Save Term score decline per Topic**

In [116]:
def save_term_rank(model, label, topics=range(0, 20, 1)):
  fig = model.visualize_term_rank(topics=topics, width=800, height=500)

  # Set the path to save 
  dir = os.path.join(WORK_DIR, COLNAME, 'term_ranks')

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(dir):
    os.makedirs(dir)

  fig.write_image(os.path.join(dir, f"term_score_trial_{label}.png"), format="png")
  fig.write_html(os.path.join(dir, f"term_score_trial_{label}.html"))

In [89]:
save_term_rank(topic_model, COLNAME, topic_range)

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/term_socore_decline_topic/tsdp_preprocessed_lemma.png?raw=true)

### **Visualize documents**

This plot shows documents and their topics in 2D

In [100]:
fig = topic_model.visualize_documents(documents, topics=range(0, 20, 1), width=800, height=700, hide_document_hover=True)
fig

**Save documents and their topics**

In [117]:
def save_documents(model, docs, label, topics=range(0, 20, 1)):
  fig = topic_model.visualize_documents(docs, topics=range(0, 20, 1), width=1000, height=800, hide_document_hover=True)

  # Set the path to save 
  dir = os.path.join(WORK_DIR, COLNAME, 'documents_topics')

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(dir):
    os.makedirs(dir)

  fig.write_image(os.path.join(dir, f"document_trial_{label}.png"), format="png")
  fig.write_html(os.path.join(dir, f"document_trial_{label}.html"))

In [106]:
save_documents(topic_model, documents, COLNAME, topic_range)

### **Term search**

In [107]:
# Find topics that contains blackfriday term
similar_topics, similarity = topic_model.find_topics("blackfriday", top_n=5)

# Show similar topics
similar_topics

[35, 75, 81, 80, 88]

In [108]:
# Show a specific topic
topic_model.get_topic(35)

[('promoção', 0.030642864540442797),
 ('black', 0.029560774705428186),
 ('cancelar', 0.020262468052136533),
 ('cupom', 0.01738332947698097),
 ('blackfriday', 0.015827596479552498),
 ('friday', 0.014625117088174468),
 ('compra', 0.013596113663886338),
 ('produto', 0.013568715150168748),
 ('loja', 0.01307238853602602),
 ('desconto', 0.013048330437276932)]

### **"Hiperparameter optimization"**

In [109]:
def save_hyperparameters(trial_params, label):
  # Set the path to save 
  dir = os.path.join(WORK_DIR, COLNAME, 'models', 'hyperparameters')

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(dir):
    os.makedirs(dir)

  with open(os.path.join(dir, f"hyperparameters_trial_{label}.json"), "w") as f:
    f.write(json.dumps(trial_params))

In [110]:
def save_model(model, label):
  # Set the path to save 
  dir = os.path.join(WORK_DIR, COLNAME, 'models')

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(dir):
    os.makedirs(dir)

  model.save(os.path.join(dir, label))

In [111]:
def save_coherence(model, topics, docs, label, clustering_model, reduction_model):
  
  # compute coherence score for BERTopic
  coherence_score = get_bertopic_coherence(model, topics, docs)
  print(f"Bertopic Coeherence score: {coherence_score}")

  # save scores
  dir = os.path.join(WORK_DIR, COLNAME, 'coherences')
  
  if os.path.exists(dir):
    os.removedirs(dir)
    os.makedirs(dir)

  with open(os.path.join(dir, 'coherence_scores.csv'), 'a', newline='') as f:
    fieldnames = ['model', 'clustering', 'reduction', 'coherence_score']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    data = [{'model': label,
             'clustering': clustering_model, 
             'reduction': reduction_model,
             'coherence_score': round(coherence_score, 4)}]
    writer.writeheader()
    writer.writerows(data)
  return coherence_score

In [142]:
# define the number of models to generate by optuna
NUMBER_OF_MODELS = 10

In [149]:
def optimizer(trial):

  clustering_option = trial.suggest_categorical('clustering_algorithm__name', ['HDBSCAN', 'K-means'])
  dimensionality_option = trial.suggest_categorical('reduction_algorithm__name', ['UMAP', 'PCA'])

  # BERTopic hyperparameters
  top_n_words = trial.suggest_int('bertopic__top_n_words', 5, 14)
  n_gram_range = ast.literal_eval(trial.suggest_categorical('bertopic__n_gram_range', ['(1,1)', '(1,2)', '(1,3)', '(2,2)', '(3,3)']))
  min_topic_size = trial.suggest_int('bertopic__min_topic_size', 15, 20)
  diversity = trial.suggest_float('bertopic__diversity', 0.1, 1.0)
  outlier_threshold = trial.suggest_float('bertopic__outliers_threshold', 0.04, 0.09)
  nr_topics = trial.suggest_int('bertopic__nr_topics', 6, 20, step=1)

  if clustering_option == 'HDBSCAN':
    # HDBSCAN hyperparameters
    min_cluster_size = trial.suggest_int('hdbscan__min_cluster_size', 10, 14)
    cluster_selection_epsilon = trial.suggest_float('hdbscan__cluster_selection_epsilon', 0.1, 1.0)

    # create a new HDBSCAN model to cluster documents
    clustering_model = HDBSCAN(min_cluster_size=min_cluster_size, prediction_data=True)
  elif clustering_option == 'K-means':
    # K-means hyperparameters
    k_means_n_clusters = trial.suggest_int('k-means__n_cluster',6, 10)

    # create a new HDBSCAN model to cluster documents
    clustering_model = KMeans(n_clusters=k_means_n_clusters, random_state=42)


  if dimensionality_option == 'UMAP':
    
    # UMAP hyperparameters
    n_neighbors = trial.suggest_int('umap__n_neighbors', 8, 12 )
    n_components = trial.suggest_int('umap__n_components', 2, 8)
    metric = trial.suggest_categorical('umap__metric', ['cosine', 'euclidean'])
    min_dist = trial.suggest_float('umap__min_dist', 0.1, 1.0)
    spread = trial.suggest_float('umap__spread', 0.1, 1.0)

    # create a new UMAP model to reduce dimension
    reduction_model = UMAP(n_neighbors=n_neighbors, metric=metric, random_state=42)
  elif dimensionality_option == 'PCA':
    
    # PCA hyperparameters
    pca_n_components = trial.suggest_int('pca__n_components', 4, 6)
    
    # create a new PCA model to reduce dimension
    reduction_model = PCA(n_components=pca_n_components, random_state=42) # k-Means, that does not produce any outliers at all

  # CountVectorizer hyperparameters 
  max_features = trial.suggest_int('vectorizer__max_features', 3000, 6000)

  # reduce the impact of frequent words.
  ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

  # create a new CountVectorizer to create a matrix of tokens count
  vectorizer_model = CountVectorizer(max_features=max_features)

  # create a new BERTopic model using multilingual option
  model = BERTopic(language="multilingual", 
                   nr_topics=nr_topics,
                   calculate_probabilities=True, 
                   verbose=False,
                   top_n_words=top_n_words,
                   n_gram_range=n_gram_range,
                   min_topic_size=min_topic_size,
                   diversity=diversity,
                   umap_model=reduction_model,
                   hdbscan_model=clustering_model,
                   vectorizer_model=vectorizer_model,
                   ctfidf_model=ctfidf_model)
    
  label = trial.number
  params = trial.params
  
  # define model id
  model_id = f"model_trial_{label}"

  # train BERTopic model 
  topics, probs = model.fit_transform(documents)

  coherence_score = 0.0
  try:
    topic_range = range(0, min(len(topics), 20), 1)
    top_n_topics = len(topic_range)

    # save plots
    save_topics(model, label, top_n_topics)
    save_documents(model, documents, label, topic_range)
    save_hierarchy(model, label, top_n_topics)
    save_term_rank(model, label, topic_range)
    save_top_words_scores(model, label, top_n_topics)
  
    # save model
    save_model(model, model_id)

    # save hyperparameters
    save_hyperparameters(params, label)

    # save model coherence score
    coherence_score = save_coherence(model, 
                   topics,
                   documents,
                   model_id,
                   clustering_option,
                   dimensionality_option)
  except ValueError: # skip models that throws ValueError: zero-size array to reduction operation maximum which has no identity
    print()
  return coherence_score

In [150]:
%%time

# create a new study
study = optuna.create_study(study_name=f'BERTopic_{COLNAME}', direction='maximize')

# run the optmize function 
study.optimize(optimizer, n_trials=NUMBER_OF_MODELS, show_progress_bar=True)

# print best value and parameters
print(f'Best value {study.best_value}')
print(f'Best params: {study.best_params}')

/usr/local/lib/python3.8/dist-packages/optuna/progress_bar.py:56: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



  0%|          | 0/15 [00:00<?, ?it/s]

Bertopic Coeherence score: 0.48517417354718584
[I 2023-01-24 19:25:39,322] Trial 0 finished with value: 0.48517417354718584 and parameters: {'clustering_algorithm__name': 'HDBSCAN', 'reduction_algorithm__name': 'UMAP', 'bertopic__top_n_words': 13, 'bertopic__n_gram_range': '(1,1)', 'bertopic__min_topic_size': 16, 'bertopic__diversity': 0.44647435025917215, 'bertopic__outliers_threshold': 0.056001304639957165, 'bertopic__nr_topics': 10, 'hdbscan__min_cluster_size': 10, 'hdbscan__cluster_selection_epsilon': 0.4938800643030927, 'umap__n_neighbors': 12, 'umap__n_components': 7, 'umap__metric': 'euclidean', 'umap__min_dist': 0.9306752732938167, 'umap__spread': 0.5637080106475547, 'vectorizer__max_features': 3035}. Best is trial 0 with value: 0.48517417354718584.

[I 2023-01-24 19:25:55,340] Trial 1 finished with value: 0.0 and parameters: {'clustering_algorithm__name': 'HDBSCAN', 'reduction_algorithm__name': 'PCA', 'bertopic__top_n_words': 11, 'bertopic__n_gram_range': '(1,1)', 'bertopic__m

#### **Merge of similar topics**

In [ ]:
# TODO: select the best model and merge similar topic and save final model

#topic_model = BERTopic.load("best_model")

In [152]:
%cd /content/Topic-Modeling-Reclame-Aqui/

/content/Topic-Modeling-Reclame-Aqui


In [ ]:
!git status

In [ ]:
!git add ./results/documents

In [154]:
!git status

^C


In [ ]:
!git commit -m "Adding document column results"

In [ ]:
!git push origin master

In [ ]:
# run this command to push a new version of this notebook in case you have saved the notebook in github and it is outdate 
!git stash
!git pull
!git stash pop

In [155]:
!rm -rf /root/.ssh/